In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import scipy.stats as stats
sns.color_palette()

/Users/joaquingianantonio/tensor_flow/env2/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
 (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)]

In [2]:
pd.options.display.float_format = "{:.2f}".format

In [3]:
%%sh
wc -l ../data/raw/UNR-IDD.csv

   37412 ../data/raw/UNR-IDD.csv


In [5]:
#import data as pd dataframe
rawdata_path = "../data/raw/UNR-IDD.csv"

raw_df = pd.read_csv(rawdata_path)

In [6]:
raw_df.head()

,Switch ID,Port Number,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Packets Rx Dropped,Packets Tx Dropped,Packets Rx Errors,...,Unknown Load/Latest,Latest bytes counter,is_valid,Table ID,Active Flow Entries,Packets Looked Up,Packets Matched,Max Size,Label,Binary Label
0,of:000000000000000c,Port#:1,132,9181,6311853,238,46,0,0,0,...,0,0,True,0,9,767,688,-1,TCP-SYN,Attack
1,of:000000000000000c,Port#:2,187,6304498,15713,171,46,0,0,0,...,0,0,True,0,9,767,688,-1,TCP-SYN,Attack
2,of:000000000000000c,Port#:3,235,6311567,8030,58,46,0,0,0,...,0,0,True,0,9,767,688,-1,TCP-SYN,Attack
3,of:000000000000000c,Port#:4,59,7878,16439,182,46,0,0,0,...,0,0,True,0,9,767,688,-1,TCP-SYN,Attack
4,of:000000000000000a,Port#:1,188,6304547,16497,183,46,0,0,0,...,0,0,True,0,7,489,403,-1,TCP-SYN,Attack


In [10]:
# Numerical and categorical columns
num_cols = list(raw_df._get_numeric_data().columns)
categ_cols = [colum for colum in raw_df.columns if colum not in num_cols]  #easy way of getting the categorical column names
print("categorical",
categ_cols , "\n"
"numerical",
num_cols)

categorical ['Switch ID', 'Port Number', 'Label', 'Binary Label'] 
numerical ['Received Packets', 'Received Bytes', 'Sent Bytes', 'Sent Packets', 'Port alive Duration (S)', 'Packets Rx Dropped', 'Packets Tx Dropped', 'Packets Rx Errors', 'Packets Tx Errors', 'Delta Received Packets', 'Delta Received Bytes', 'Delta Sent Bytes', 'Delta Sent Packets', 'Delta Port alive Duration (S)', 'Delta Packets Rx Dropped', ' Delta Packets Tx Dropped', 'Delta Packets Rx Errors', 'Delta Packets Tx Errors', 'Connection Point', 'Total Load/Rate', 'Total Load/Latest', 'Unknown Load/Rate', 'Unknown Load/Latest', 'Latest bytes counter', 'is_valid', 'Table ID', 'Active Flow Entries', 'Packets Looked Up', 'Packets Matched', 'Max Size']


In [12]:
raw_df.columns

Index(['Switch ID', 'Port Number', 'Received Packets', 'Received Bytes',
       'Sent Bytes', 'Sent Packets', 'Port alive Duration (S)',
       'Packets Rx Dropped', 'Packets Tx Dropped', 'Packets Rx Errors',
       'Packets Tx Errors', 'Delta Received Packets', 'Delta Received Bytes',
       'Delta Sent Bytes', 'Delta Sent Packets',
       'Delta Port alive Duration (S)', 'Delta Packets Rx Dropped',
       ' Delta Packets Tx Dropped', 'Delta Packets Rx Errors',
       'Delta Packets Tx Errors', 'Connection Point', 'Total Load/Rate',
       'Total Load/Latest', 'Unknown Load/Rate', 'Unknown Load/Latest',
       'Latest bytes counter', 'is_valid', 'Table ID', 'Active Flow Entries',
       'Packets Looked Up', 'Packets Matched', 'Max Size', 'Label',
       'Binary Label'],
      dtype='object')

In [13]:
raw_df.shape

(37411, 34)

In [14]:
# create reduced dataset without columns that provide no information.
# dropping also switch  id and port id

reduced_df = raw_df[[#'Switch ID', 
                        #'Port Number', 
                        'Received Packets', 
                        'Received Bytes', 
                        'Sent Bytes', 
                        'Sent Packets', 
                        'Port alive Duration (S)',
                        #'Packets Rx Dropped', 
                        #'Packets Tx Dropped', 
                        #'Packets Rx Errors',
                        #'Packets Tx Errors', 
                        'Delta Received Packets', 
                        'Delta Received Bytes',
                        'Delta Sent Bytes', 
                        'Delta Sent Packets',
                        'Delta Port alive Duration (S)', 
                        #'Delta Packets Rx Dropped',
                        #' Delta Packets Tx Dropped', 
                        #'Delta Packets Rx Errors',
                        #'Delta Packets Tx Errors', 
                        'Connection Point', 
                        'Total Load/Rate',
                        'Total Load/Latest', 
                        'Unknown Load/Rate', 
                        'Unknown Load/Latest',
                        'Latest bytes counter', 
                        #'is_valid', 
                        #'Table ID', 
                        'Active Flow Entries',
                        'Packets Looked Up', 
                        'Packets Matched', 
                        #'Max Size', 
                        'Label',
                        'Binary Label']]

In [15]:
reduced_df.shape

(37411, 21)

In [16]:
# save reduced dataset

reduced_df.to_csv("../data/processed/reduced_df.csv")

In [28]:
# create the labels for binary labels and label

labels_binlab = reduced_df["Binary Label"]
labels_labcat = reduced_df["Label"]

labels_binlab.head(), labels_binlab.shape, labels_labcat.head(), labels_labcat.shape

(0    Attack
 1    Attack
 2    Attack
 3    Attack
 4    Attack
 Name: Binary Label, dtype: object,
 (37411,),
 0    TCP-SYN
 1    TCP-SYN
 2    TCP-SYN
 3    TCP-SYN
 4    TCP-SYN
 Name: Label, dtype: object,
 (37411,))

In [29]:
# encode the labels with values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

labels_binlab = labels_binlab.replace(to_replace= ["Attack", "Normal"], value= [1 , 0])
labels_labcat = labels_labcat.replace(to_replace= ['Blackhole', 'Diversion', 'Normal', 'Overflow', 'PortScan', 'TCP-SYN'], value= [1 , 2, 3, 4, 5, 6])

labels_binlab.unique(), labels_labcat.unique()

(array([1, 0]), array([6, 1, 2, 4, 3, 5]))

In [32]:
ylab = le.fit_transform(labels_binlab)
ycat = le.fit_transform(labels_labcat)

ylab[:20] , ycat[:20]

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]))

In [19]:
# drop the label columns creating the reduced features dataframe
r_features = reduced_df.drop(['Label', "Binary Label"], axis=1)

r_features.shape


(37411, 19)

In [30]:
# save all new dataframes

r_features.to_csv("../data/processed/r_features.csv")
labels_binlab.to_csv("../data/processed/labels_binlab.csv")
labels_labcat.to_csv("../data/processed/labels_labcat.csv")

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [34]:
# separate data in train and test sets, using binlab labels.
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(r_features,labels_binlab, random_state= 0 , test_size= 0.2)


In [48]:
# save the test and train sets
sets = [X_train_bin, X_test_bin, y_train_bin, y_test_bin]
names = list(str("X_train_bin,X_test_bin,y_train_bin,y_test_bin").split(','))
for idx in range(len(sets)):
    sets[idx].to_csv("../data/processed/" + names[idx] + ".csv")
